In [ ]:
# -*- coding: utf-8 -*-
"""

@author: Parmita Ghosh

Synergy of optical and synthetic aperture radar (SAR) data for early-stage crop yield estimation: A case study over a state of Germany
Methodology:
Step 1: Baseline Random Forest Regression Model
In this notebook we are importing ground truth yield data set along with Optical and SAR image features. 70% of the data is being used for taring and 30% for testing the model. 
First a baseline random forest regression model with tuned hyper-paramets is being developed with all the input image features.

Step2: The performance of baseline model is being evaluated by correlation coeffifient (r), Mean Absolute Error (MAE) and Root Mean Square Error (RMSE) of obsered yield and predicted yield

Step 3: An omtimised random forest regression with genetic algorithm (GA) based feature selection is being developed. 
This GA feature selection algorithm selected the best set of input image features for yied estimation with random forest regression model.

Step 4: The performance of omtimised random forest regression with genetic algorithm (GA) based feature selection is being evaluated by correlation coeffifient (r), Mean Absolute Error (MAE) and Root Mean Square Error (RMSE) of obsered yield and predicted yield

Step 5: Visualisation of baseline and optimised model performance


"""

######################################################################  Step 1: Baseline Random Forest Regression Model ####################################
######################
######################################################   Data Preparation ##############
# Pandas is used for data manipulation
import pandas as pd
features=pd.read_csv('WinterWheat_SampleData.csv')#Reading the Winter Wheat Ground Truth data in csv Format  
features.head(5) #Display first 5 rows


# Use numpy to convert to arrays
import numpy as np

labels = np.array(features['Yield'])# Labels are the Crop Yield values
features= features.drop('Yield', axis = 1)# Remove the Crop Yield from the features
feature_list = list(features.columns)# Saving feature names for later use
features = np.array(features)# Convert features to numpy array

############################################## Tarin, Test data Preparation #########################

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.30, random_state = 42)# Split the data into 70% training and 30% testing sets

###################################### Baseline Random Forest Model Development and Hyper-Parameters Tuning with GridSearchCV#######################

#Importing required libraries
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

rfr=RandomForestRegressor(random_state=42)

param_grid = {
    'bootstrap': [True],
    'max_depth': [int(x) for x in np.linspace(10,100,10,endpoint=True,dtype = int)],
    'min_samples_leaf': [int(x) for x in np.linspace(2,10,9,endpoint=True,dtype = int)],
    'min_samples_split':[int(x) for x in np.linspace(2,15,14,endpoint=True,dtype = int)],
    'n_estimators': [int(x) for x in np.linspace(50,1200,24,endpoint=True,dtype = int)]
} #Grid Space for Hyper-parameters tuning


CV_rfc = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 10,scoring='neg_mean_absolute_error',n_jobs = -1, verbose = 2)
CV_rfc.fit(train_features,train_labels)# Grid Search with 10-fold Cross Validation for Hyper-parameter tuning
####################Display Best set of Hyper-parameter 
print(CV_rfc.best_params_)
##########Save the baseline random forest regression model with the results of hyper-parametrs tuning (CV_rfc.best_params_) 
BaseLineRFR=RandomForestRegressor(bootstrap=True,
                                             max_depth=80,
                                             min_samples_leaf=5,
                                             min_samples_split=8,
                                             n_estimators=100)
import joblib

joblib.dump(BaseLineRFR, "./BaseLineModelrandom_forest_Regression.joblib",compress=3)# Saving the Baseline model for future use


###################################################### Step2: The performance Evaluation of baseline Random Forest Regression Model #################################

BaseLineModel = joblib.load("./BaseLineModelrandom_forest_Regression.joblib")# load the baseline model 
BaseLineModel.fit(train_features,train_labels)
Bpredicted_labels_train_features=BaseLineModel.predict(train_features)#Predicting yield with training dataset
Bpredicted_labels_test_features=BaseLineModel.predict(test_features)#Predicting yield with testing dataset

############################################# Baseline Random Forest Regression Yield Model Performance Evaluation

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print("Correlation Coefficient (r) of Baseline Random Forest Regression Model on Training Data: ",pearsonr(train_labels,Bpredicted_labels_train_features))
print("Correlation Coefficient (r) of Baseline Random Forest Regression Model on Testing Data: ",pearsonr(test_labels,Bpredicted_labels_test_features))

print("MAE of Baseline Random Forest Regression Model on Training Data: ",mean_absolute_error(train_labels,Bpredicted_labels_train_features))###t/ha
print("MAE of Baseline Random Forest Regression Model on Testing Data: ",mean_absolute_error(test_labels,Bpredicted_labels_test_features))###t/ha

print("RMSE of Baseline Random Forest Regression Model on Training Data: ",np.sqrt(mean_squared_error(train_labels,Bpredicted_labels_train_features)))###t/ha
print("RMSE of Baseline Random Forest Regression Model on Testing Data: ",np.sqrt(mean_squared_error(test_labels,Bpredicted_labels_test_features)))###t/ha

###################################################################################################################################################################
##############################      Step 3: Opmtimised random forest regression with genetic algorithm (GA) based feature selection  ################

from sklearn import linear_model
from genetic_selection import GeneticSelectionCV

                                        
Featureselector = GeneticSelectionCV(BaseLineModel,
                                  cv=5,
                                  verbose=1,
                                  scoring="neg_mean_absolute_error", 
                                  max_features=20,
                                  n_population=20,
                                  crossover_proba=0.05,
                                  mutation_proba=0.001,
                                  n_generations=50,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)#Feature Selection with Genetic Algorithm with Baseline Random ForestRegressor estimator with stopping criteria as NEGATIVE MSE
FeatureselectorModel = Featureselector.fit(train_features,train_labels)#Fitting the training data into Opmtimised random forest regression with genetic algorithm (GA) based feature selection

######################### Visualise the results of feature Selection ################
featurename= list(features..columns.values)#List of Input Feature Names
df = pd.DataFrame((featurename,Featureselector.support_,Featureselector.generation_scores_))### Feature Selection Result

Transpose=df.T
Transpose.columns =['Feature','Support','Score']
Transpose.head()#Showing the Selected Features
import seaborn as sns
import matplotlib.pyplot as plt
sns.factorplot(x= 'Feature', y= 'Support', data= Transpose, kind='bar', legend='True')
plt.title('Feature Selection',fontsize=15)
plt.show()#Ploting the selected features based on the Featureselector.support_


##############################  Step 4: Performance evaluation of omtimised random forest regression with genetic algorithm (GA) based feature selection ############
predicted_labels_train_features=FeatureselectorModel .predict(train_features)
predicted_labels_test_features=FeatureselectorModel .predict(test_features)

print("Correlation Coefficient (r) of Optimised Random Forest Regression Model on Training Data: ",pearsonr(train_labels,predicted_labels_train_features))
print("Correlation Coefficient (r) of Optimised Random Forest Regression Model on Testing Data: ",pearsonr(test_labels,predicted_labels_test_features))

print("MAE of Optimised Random Forest Regression Model on Training Data: ",mean_absolute_error(train_labels,predicted_labels_train_features))#t/ha
print("MAE of Optimised Random Forest Regression Model on Testing Data: ",mean_absolute_error(test_labels,predicted_labels_test_features))#t/ha

print("RMSE of Optimised Random Forest Regression Model on Training Data: ",np.sqrt(mean_squared_error(train_labels,predicted_labels_train_features)))#t/ha
print("RMSE of Optimised Random Forest Regression Model on Testning Data: ",np.sqrt(mean_squared_error(test_labels,predicted_labels_test_features)))#t/ha

################      Step 5: Visualisation of baseline and optimised model performance ############
fig = plt.figure(figsize=(15,5))
######################################  Baseline Model##########################################
ax1 = plt.subplot(1, 2, 1)
ax1.scatter(train_labels,Bpredicted_labels_train_features,s=10, c='b', marker="o", label="Training Dataset = %.0f"%(train_labels.shape))
ax1.scatter(test_labels,Bpredicted_labels_test_features,s=10, c='r', marker="o", label="Testing Dataset = %.0f"%(test_labels.shape))
plt.xlabel('Observed Yield (t/ha)',fontsize=15)
plt.ylabel('Predicted Yield(t/ha)',fontsize=15)
plt.title('Baseline Model',fontsize=15)
plt.legend(loc="lower right")
############################ Optimised Model ###############
ax1 = plt.subplot(1, 2, 2)
ax1.scatter(train_labels,predicted_labels_train_features,s=10, c='b', marker="o",label="Training Dataset = %.0f"%(train_labels.shape))
ax1.scatter(test_labels,predicted_labels_test_features,s=10, c='r', marker="o", label="Testing Dataset = %.0f"%(test_labels.shape))
plt.xlabel('Observed Yield (t/ha)',fontsize=15)
plt.ylabel('Predicted Yield(t/ha)',fontsize=15)
plt.title('Optimised Model',fontsize=15)
plt.legend(loc="lower right")
plt.show()